In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
import requests
import csv
import time
import pyarrow.parquet as pq

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Start a webdriver session
driver = webdriver.Firefox()

# List to store all product information
all_data = []

# Loop through two pages of results
for page in range(0, 2):
    # Navigate to page
    driver.get(f'https://www.amazon.fr/s?i=computers&rh=n%3A429879031&fs=true&page=2&qid=1675780683&ref=sr_pg_{page}')
    time.sleep(1)

    # Handle cookie dialog
    try:
        cookie_button = driver.find_element(By.CLASS_NAME, 'a-link-emphasis')
        cookie_button.click()
    except:
        # Get product links from the page
        product_links = [x.get_attribute('href') for x in driver.find_elements(By.XPATH,('//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]'))]

        # Loop through product links
        for link in product_links:
            # Navigate to product page
            driver.get(link)
            time.sleep(1)

            # Extract product information
            product = driver.find_elements(By.ID,"dp-container")[0]

            # Helper function to extract data
            def extract_data(xpath):
                try:
                    data = product.find_element(By.XPATH, xpath).get_attribute('textContent')
                except:
                    data = " "
                return data

            # Basic product information
            name = extract_data('//span[@class="a-size-large product-title-word-break"]')
            price = extract_data('//span[@class="a-price-whole"]')
            rating = extract_data('//span[@class="a-icon-alt"]')

            # Product details in a table
            try:
                print("test 0")
                WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//table[@class="a-keyvalue prodDetTable"]/tbody/tr')))
            except TimeoutException:
                print("TableNotFound")
            else:
                table = driver.find_element(By.XPATH, '//table[@class="a-keyvalue prodDetTable"]')
                tr_elements = table.find_elements(By.XPATH, 'tbody/tr')
    # Dictionary to store product details
                product_details = {}
                for tr in tr_elements:
                    th_elements = tr.find_elements(By.XPATH, 'th[@class="a-color-secondary a-size-base prodDetSectionEntry"]')
                    td_elements = tr.find_elements(By.XPATH, 'td[@class="a-size-base prodDetAttrValue"]')
                    for th, td in zip(th_elements, td_elements):
                        product_details[th.get_attribute('textContent').strip('\n')] = td.get_attribute('textContent').strip('\n')
                        print("test 3")
               
            # Add all product information to the list
            all_data.append({"Name": name, "Price": price, "Rating": rating, **product_details})

# Convert list of dictionaries to a pandas DataFrame
df = pd.DataFrame(all_data)

# Store DataFrame in a Parquet file
df.to_parquet('dfAmazon.parquet')

# Close the webdriver session
driver.quit()

38
['https://www.amazon.fr/sspa/click?ie=UTF8&spc=MToyNTAyNTAwMjQzNjM4OTI0OjE2NzU4MDE2NDc6c3BfYXRmX2Jyb3dzZToyMDExMTEwMzEzODU5ODo6MDo6&url=%2FHP-Chromebook-14c-cc0000sf-Tactile-i3-11xxx%2Fdp%2FB095ND633K%2Fref%3Dsr_1_25_sspa%3Fqid%3D1675801647%26s%3Dcomputers%26sr%3D1-25-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl%26psc%3D1', 'https://www.amazon.fr/sspa/click?ie=UTF8&spc=MToyNTAyNTAwMjQzNjM4OTI0OjE2NzU4MDE2NDc6c3BfYXRmX2Jyb3dzZToyMDAxNDQ0NjY2NTg5ODo6MDo6&url=%2FJumper-Ordinateur-Portable-bi-Bande-lextension%2Fdp%2FB09D7B1JJ4%2Fref%3Dsr_1_26_sspa%3Fqid%3D1675801647%26s%3Dcomputers%26sr%3D1-26-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl%26psc%3D1', 'https://www.amazon.fr/sspa/click?ie=UTF8&spc=MToyNTAyNTAwMjQzNjM4OTI0OjE2NzU4MDE2NDc6c3BfYXRmX2Jyb3dzZToyMDA2MTQ5MDQyNTE5ODo6MDo6&url=%2FOrdinateur-Portable-3000x2000-r%25C3%25A9tro%25C3%25A9clair%25C3%25A9-empreinte%2Fdp%2FB09YYDKQK5%2Fref%3Dsr_1_27_sspa%3Fqid%3D1675801647%26s%3Dcomputers%26sr%3D1-27-spons%26sp_csd%3Dd2lkZ2V0TmFt

TypeError: 'WebElement' object is not iterable

In [15]:
df = pd.read_parquet('dfAmazon.parquet', engine='pyarrow')
df

,Name,Price,Rating,Marque,Numéro du modèle de l'article,séries,Garantie constructeur,Système d'exploitation,Plate-forme du matériel informatique,\tDescription du clavier\t,...,Dimensions de l'article L x L x H,Poids du produit,\tVitesse de rotation du disque dur\t,Nombre de ports USB 2.0,Étirement,Nombre de ports Ethernet,Bluetooth,Compatibilité du périphérique,Ecran,Caractéristiques spéciales
0,ALLDOCUBE GTBook 15 Ordinateur Portabl...,"399,","3,8 sur 5 étoiles",‎ALLDOCUBE,‎i1502-ODLF-EU,‎I1502-ODLF-EU,‎2 year,‎Windows 11,‎PC,‎French keyboard,...,None,None,None,None,None,None,None,None,None,None
1,"Jumper Ordinateur Portable 13,3 Pouces...","269,","4,1 sur 5 étoiles",‎Jumper,‎X3464,‎EZbook X3 office,‎1 Year Warranty Against Manuf...,‎Windows 10,‎Windows,‎Fr,...,‎31.5 x 20.8 x 1.8 centimètres,‎1.23 Kilogrammes,None,None,None,None,None,None,None,None
2,"Ordinateur Portable 12,5"" Windows 11, ...","309,","2,2 sur 5 étoiles",‎Morostron,‎1257G,‎‎1257G,None,‎Windows 11,‎PC,‎French keyboard,...,None,‎1060 Grammes,‎2400,None,None,None,None,None,None,None
3,ASUS Vivobook 14 R415JA-EB2111W Ordina...,"599,","4,2 sur 5 étoiles",‎ASUS,‎R415JA-EB2111W,‎Vivobook,‎2 year manufacturer,‎Windows 11 Home,‎PC,‎AZERTY + NUMPAD,...,‎32.5 x 2 x 21.6 centimètres,‎1.6 Kilogrammes,None,‎2,None,None,None,None,None,None
4,Lenovo IdeaPad 1 15IJL7 Ordinateur Por...,"499,","4,3 sur 5 étoiles",‎Lenovo,‎82LX001TFR,‎Notebook,‎2 ans,‎Windows 11,None,‎French keyboard,...,‎36 x 1.8 x 23.6 centimètres,‎1.55 Kilogrammes,None,‎1,None,None,None,None,None,None
5,HP Laptop 17-cp0001sf PC Portable 17.3...,"899,","4,4 sur 5 étoiles",‎HP,‎64V29EA#ABF,‎64V29EA,‎Cette garantie limitée valabl...,‎Windows 11 Home,None,‎French keyboard,...,‎25.8 x 40.1 x 2 centimètres,‎2070 Grammes,None,None,None,None,None,None,None,None
6,"Jumper Ordinateur Portable Windows 11,...","299,","5,0 sur 5 étoiles",‎Jumper,‎EZbook S5,‎EZbook S5,‎1 an,‎Windows 11 Home,None,None,...,None,‎1.25 Kilogrammes,None,‎1,None,None,None,None,None,None
7,Acer Aspire 3 A314-35-P6EC Ordinateur ...,"299,","3,9 sur 5 étoiles",‎Acer,‎NX.ACGEF.006,‎Aspire A314-35-P6EC,‎2 ans constructeur,‎Win 11 Home en mode S,None,‎Traditionnel,...,None,‎1.45 Kilogrammes,None,‎1,None,None,None,None,None,None
8,Apple 2022 MacBook Air avec Puce M2 : ...,"1 349,","4,7 sur 5 étoiles",‎Acer,‎NX.ACGEF.006,‎Aspire A314-35-P6EC,‎2 ans constructeur,‎Win 11 Home en mode S,None,‎Traditionnel,...,None,‎1.45 Kilogrammes,None,‎1,None,None,None,None,None,None
9,Aocwei Ordinateur Portable 128Go SSD S...,"223,","4,4 sur 5 étoiles",‎Aocwei,‎A5,‎A5,‎1,‎Windows 10,‎Windows,‎Jeu,...,None,‎1.32 Kilogrammes,None,None,None,None,None,None,None,None


In [16]:
df.head(10)


,Name,Price,Rating,Marque,Numéro du modèle de l'article,séries,Garantie constructeur,Système d'exploitation,Plate-forme du matériel informatique,\tDescription du clavier\t,...,Dimensions de l'article L x L x H,Poids du produit,\tVitesse de rotation du disque dur\t,Nombre de ports USB 2.0,Étirement,Nombre de ports Ethernet,Bluetooth,Compatibilité du périphérique,Ecran,Caractéristiques spéciales
0,ALLDOCUBE GTBook 15 Ordinateur Portabl...,"399,","3,8 sur 5 étoiles",‎ALLDOCUBE,‎i1502-ODLF-EU,‎I1502-ODLF-EU,‎2 year,‎Windows 11,‎PC,‎French keyboard,...,None,None,None,None,None,None,None,None,None,None
1,"Jumper Ordinateur Portable 13,3 Pouces...","269,","4,1 sur 5 étoiles",‎Jumper,‎X3464,‎EZbook X3 office,‎1 Year Warranty Against Manuf...,‎Windows 10,‎Windows,‎Fr,...,‎31.5 x 20.8 x 1.8 centimètres,‎1.23 Kilogrammes,None,None,None,None,None,None,None,None
2,"Ordinateur Portable 12,5"" Windows 11, ...","309,","2,2 sur 5 étoiles",‎Morostron,‎1257G,‎‎1257G,None,‎Windows 11,‎PC,‎French keyboard,...,None,‎1060 Grammes,‎2400,None,None,None,None,None,None,None
3,ASUS Vivobook 14 R415JA-EB2111W Ordina...,"599,","4,2 sur 5 étoiles",‎ASUS,‎R415JA-EB2111W,‎Vivobook,‎2 year manufacturer,‎Windows 11 Home,‎PC,‎AZERTY + NUMPAD,...,‎32.5 x 2 x 21.6 centimètres,‎1.6 Kilogrammes,None,‎2,None,None,None,None,None,None
4,Lenovo IdeaPad 1 15IJL7 Ordinateur Por...,"499,","4,3 sur 5 étoiles",‎Lenovo,‎82LX001TFR,‎Notebook,‎2 ans,‎Windows 11,None,‎French keyboard,...,‎36 x 1.8 x 23.6 centimètres,‎1.55 Kilogrammes,None,‎1,None,None,None,None,None,None
5,HP Laptop 17-cp0001sf PC Portable 17.3...,"899,","4,4 sur 5 étoiles",‎HP,‎64V29EA#ABF,‎64V29EA,‎Cette garantie limitée valabl...,‎Windows 11 Home,None,‎French keyboard,...,‎25.8 x 40.1 x 2 centimètres,‎2070 Grammes,None,None,None,None,None,None,None,None
6,"Jumper Ordinateur Portable Windows 11,...","299,","5,0 sur 5 étoiles",‎Jumper,‎EZbook S5,‎EZbook S5,‎1 an,‎Windows 11 Home,None,None,...,None,‎1.25 Kilogrammes,None,‎1,None,None,None,None,None,None
7,Acer Aspire 3 A314-35-P6EC Ordinateur ...,"299,","3,9 sur 5 étoiles",‎Acer,‎NX.ACGEF.006,‎Aspire A314-35-P6EC,‎2 ans constructeur,‎Win 11 Home en mode S,None,‎Traditionnel,...,None,‎1.45 Kilogrammes,None,‎1,None,None,None,None,None,None
8,Apple 2022 MacBook Air avec Puce M2 : ...,"1 349,","4,7 sur 5 étoiles",‎Acer,‎NX.ACGEF.006,‎Aspire A314-35-P6EC,‎2 ans constructeur,‎Win 11 Home en mode S,None,‎Traditionnel,...,None,‎1.45 Kilogrammes,None,‎1,None,None,None,None,None,None
9,Aocwei Ordinateur Portable 128Go SSD S...,"223,","4,4 sur 5 étoiles",‎Aocwei,‎A5,‎A5,‎1,‎Windows 10,‎Windows,‎Jeu,...,None,‎1.32 Kilogrammes,None,None,None,None,None,None,None,None


In [12]:
df.shape
df.isnull().sum()

Name                                             0
Price                                            0
Rating                                           0
 Marque                                          0
 Numéro du modèle de l'article                   1
 séries                                          2
 Garantie constructeur                           5
 Système d'exploitation                          0
 Plate-forme du matériel informatique           17
 \tDescription du clavier\t                      7
 Marque du processeur                            1
 Type de processeur                              1
 Vitesse du processeur                           4
 Nombre de coeurs                                1
 Mémoire maximale                                2
 Taille du disque dur                           14
 Technologie du disque dur                       1
 Interface du disque dur                        15
 Taille de l'écran                               4
 Résolution de l'écran         

test 0
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 0
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 0
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 0
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 3
test 0
test 3
test 3
test 3
test 3